In [1]:
!git status

On branch master
Your branch is up-to-date with 'origin/master'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.ipynb_checkpoints/
	Seq2Seq-Beijing-Learn.ipynb

nothing added to commit but untracked files present (use "git add" to track)


In [2]:
!git pull origin master

From https://github.com/Adrian-Gordon/seq2seq-sagemaker
 * branch            master     -> FETCH_HEAD
Already up-to-date.


In [3]:
from sagemaker import get_execution_role

import boto3

import json

role = get_execution_role()

bucket='culturehub'

#get the data
data_key = 'seqtoseq/PRSA_data_2010.1.1-2014.12.31.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

#get the configuration

configuration_key ='seqtoseq/config_beijing.json'
configuration_location = 's3://{}/{}'.format(bucket, configuration_key)

configuration_string = boto3.resource('s3').Object(bucket, configuration_key).get()['Body'].read().decode('utf-8')

print("Configuration:")
print(configuration_string)

config = json.loads(configuration_string)


Configuration:
{
"data_modulename":"generate_beijing_data",
"datafilename": "./data/PRSA_data_2010.1.1-2014.12.31.csv",
"savefilename":"beijing-seq2seq0",
"input_sequence_length": 30,
"output_sequence_length": 5,
"batch_size" :16,
"input_dim" : 11,
"output_dim":1,
"num_stacked_layers" :2,
"hidden_dim" :64,
"learning_rate" :0.01,
"l2_regularization_lambda":0.03,
"gradient_clipping": 2.5,
"epochs": 100
}


In [4]:
#get the data

from generatedata import *

bucket='culturehub'
data_key = 'seqtoseq/PRSA_data_2010.1.1-2014.12.31.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)
gd=GenerateData(data_location)
print(gd.data.head())



s3://culturehub/seqtoseq/PRSA_data_2010.1.1-2014.12.31.csv
   No  year  month  day  hour     pm2.5      DEWP      TEMP      PRES  \
0   1  2010      1    1     0 -1.018534 -1.580878 -1.922250  0.443328   
1   2  2010      1    1     1 -1.018534 -1.580878 -2.004228  0.345943   
2   3  2010      1    1     2 -1.018534 -1.580878 -1.922250  0.248559   
3   4  2010      1    1     3 -1.018534 -1.580878 -2.168183  0.248559   
4   5  2010      1    1     4 -1.018534 -1.511594 -2.004228  0.151174   

        Iws        Is        Ir  cbwd_NE  cbwd_NW  cbwd_SE  cbwd_cv  
0 -0.441894 -0.069353 -0.137667        0        1        0        0  
1 -0.379306 -0.069353 -0.137667        0        1        0        0  
2 -0.343514 -0.069353 -0.137667        0        1        0        0  
3 -0.280926 -0.069353 -0.137667        0        1        0        0  
4 -0.218339 -0.069353 -0.137667        0        1        0        0  


In [ ]:
#learn

from seq2seqmodel import *

import os

def main():
    
    
    seq2seq  = Seq2Seq(config)

    loss_summary = tf.summary.scalar("loss", seq2seq.loss)
    merged = tf.summary.merge_all()


    init = tf.global_variables_initializer()
  
    with tf.Session() as sess:
   
        sess.run(init)

        for epoch in range(config["epochs"]):

     
            writer = tf.summary.FileWriter('./graphs/seq2seq/basic', sess.graph)

            #generate a data batch
            input_batch, output_batch = gd.getTrainingSample(batch_size = config["batch_size"], input_seq_length =config["input_sequence_length"], output_seq_length = config["output_sequence_length"])

            reshaped_input_batch = gd.reshape(np.array(input_batch),config["input_sequence_length"], config["input_dim"])

     
            reshaped_output_batch = gd.reshape(np.array(output_batch),config["output_sequence_length"], config["output_dim"])

            feed_dict={seq2seq.encoder_inputs[t]: reshaped_input_batch[t] for t in range(config["input_sequence_length"])}

            feed_dict.update({seq2seq.decoder_target_inputs[t]: reshaped_output_batch[t] for t in range(config["output_sequence_length"])})
     

            #the_decoder_outputs = sess.run(seq2seq.encode_decode,feed_dict)

            _, loss = sess.run([seq2seq.optimize, seq2seq.loss],feed_dict)

            summary = sess.run(merged,feed_dict)
            writer.add_summary(summary, epoch)
     
            print(epoch,loss)
   
        saver = tf.train.Saver
        save_path = saver().save(sess, os.path.join('./',config["savefilename"]))
        print("Checkpoint saved at: ", save_path)
    
main()

0 2.1989867471855264
1 2.851043589841268
2 6.408480212202025
3 2.218532951238478
4 1.5419816817504302
5 1.0480882908541442
6 1.519546285227316
7 1.8213039050238269
8 0.841788959575196
9 0.8471662316804788
10 0.8529783823784529
11 0.9965009154918675
12 1.3454427006791738
13 0.7795345504702291
14 0.7818845017631202
15 0.6915746839025418
16 0.8583896188935266
17 1.090032292970069
18 0.7045560892999467
19 0.7210055584257332
20 0.6400730944077393
21 0.9517587209512905
22 0.6477928814937159
23 0.6889379173328283
24 0.8524288646717323
25 0.7239621015889807
26 0.600240578222153
27 0.5351876518052611
28 0.5625681270933027
29 0.8849653208890069
30 0.5353460073226219
31 0.6028103915504657
32 0.6127935454039085
33 0.784477449675016
34 0.9891543765544969
35 0.45875740811719856
36 0.5062837651524307
37 0.8150321902297677
38 0.48822178298055474
39 0.46978266883254716
40 0.5435536966157292
41 0.4966749185579714
42 0.4520366198578858
43 0.4415711814148492
44 0.4749181625123694
45 0.5310222850421795
46 